<a href="https://colab.research.google.com/github/benjaminnigjeh/automated_data_mining/blob/main/torch_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Install additional libraries

In [ ]:
!pip Install wget
!pip install xlstm

##Import external libraries

In [ ]:
import xlstm
import wget
import h5py as h5
from random import shuffle
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from xlstm import (
    xLSTMBlockStack,
    xLSTMBlockStackConfig,
    mLSTMBlockConfig,
    mLSTMLayerConfig,
    sLSTMBlockConfig,
    sLSTMLayerConfig,
    FeedForwardConfig,
)


##Download the raw dataset

In [ ]:
# Download the dataset
url = 'https://figshare.com/ndownloader/files/12506534'
wget.download(url)

##Prepare training and test dataset compatible with pytorch

In [ ]:
# Define the constants
PROSIT_ALHABET = {"A": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "H": 7, "I": 8, "K": 9, "L": 10, "M": 11, "N": 12, "P": 13, "Q": 14, "R": 15, "S": 16, "T": 17, "V": 18, "W": 19, "Y": 20, "M(ox)": 21}
PROSIT_INDEXED_ALPHABET = {i: c for c, i in PROSIT_ALHABET.items()}

# Read from the dowloaded raw dataset
with h5.File('/content/holdout_hcd.hdf5', 'r') as f:
  KEY_ARRAY = ["sequence_integer", "precursor_charge_onehot", "intensities_raw"]
  KEY_SCALAR = ["collision_energy_aligned_normed", "collision_energy"]
  df = pd.DataFrame({key: list(f[key][...]) for key in KEY_ARRAY})
  for key in KEY_SCALAR:
    df[key] = f[key][...]

# Add convenience columns
df['precursor_charge'] = df.precursor_charge_onehot.map(lambda a: a.argmax() + 1)
df['sequence_maxquant'] = df.sequence_integer.map(lambda s: "".join(PROSIT_INDEXED_ALPHABET[i] for i in s if i != 0))
df['sequence_length'] = df.sequence_integer.map(lambda s: np.count_nonzero(s))

# Generate unique sequences
def unique_dataframe(df, unique_column):

    unique = list(set(df[unique_column]))
    n_unique = len(unique)
    shuffle(unique)
    unique_train = unique
    df_train = df[df[unique_column].isin(unique_train)]
    assert len(df_train) == len(df)
    return df_train

df_train = unique_dataframe(df, unique_column='sequence_maxquant')


# Prepare the training dataset in numpy
INPUT_COLUMNS = ('sequence_integer', 'precursor_charge_onehot', 'collision_energy_aligned_normed')
OUTPUT_COLUMN = 'intensities_raw'

x_train = [np.vstack(df_train[column]) for column in INPUT_COLUMNS]
y_train = np.vstack(df_train[OUTPUT_COLUMN])

# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define batch size
batch_size = 256

# Convert numpy arrays to torch tensors
input1 = torch.tensor(x_train[0], dtype=torch.int64)
input2 = torch.tensor(x_train[1], dtype=torch.float32)
input3 = torch.tensor(x_train[2], dtype=torch.float32)
labels = torch.tensor(y_train, dtype=torch.float32)

# Move the data to the CUDA device
input1, input2, input3, labels = input1.to(device), input2.to(device), input3.to(device), labels.to(device)

# Split the data into training and test sets
train_input1, test_input1, train_input2, test_input2, train_input3, test_input3, train_labels, test_labels = train_test_split(
    input1, input2, input3, labels, test_size=0.2, random_state=42
)

# Generate dataLoader for batching
train_dataset = torch.utils.data.TensorDataset(train_input1, train_input2, train_input3, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torch.utils.data.TensorDataset(test_input1, test_input2, test_input3, test_labels)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Transformer encoder architecture

In [ ]:
# Define the Transformer Encoder Model
class TransformerEncoderModel(nn.Module):
    def __init__(self):
        super(TransformerEncoderModel, self).__init__()

        # Update embedding layer for input1 to handle values from 0 to 21 and output 32-dimensional embeddings
        self.embedding_input1 = nn.Embedding(22, 32)  # 22 because the values range from 0 to 21

        # Transformer encoder for the first input (32-dimensional sequence), batch_first=True
        self.transformer_encoder_layer = nn.TransformerEncoderLayer(d_model=32, nhead=2, batch_first=True)
        self.transformer_encoder = nn.TransformerEncoder(self.transformer_encoder_layer, num_layers=2)

        # Fully connected layer for the first input (32-dimensional sequence)
        self.fc_input1 = nn.Linear(32, 124)

        # Fully connected layer for the second input (6-dimensional sequence)
        self.fc_input2 = nn.Linear(6, 124)

        # Fully connected layer for the third input (1-dimensional sequence)
        self.fc_input3 = nn.Linear(1, 124)

        # Fully connected layer after multiplication of both inputs
        self.fc_after_multiplication = nn.Linear(124, 124)

        # Final output layer to produce a 174-dimensional output
        self.fc_output = nn.Linear(124, 174)

    def forward(self, input1, input2, input3):
        # Process the first input through the embedding layer
        input1 = self.embedding_input1(input1.long())  # Ensure input1 is long for embedding

        # Pass the embedded input1 through the transformer encoder
        input1 = self.transformer_encoder(input1)  # [batch_size, seq_len, feature_dim]

        # Reduce input1 over the sequence length dimension (e.g., take mean)
        input1 = input1.sum(dim=1)  # Now shape is [batch_size, feature_dim]

        # Process the output of the transformer through the fully connected layer
        input1 = self.fc_input1(input1)

        # Process the second input through the fully connected layer
        input2 = self.fc_input2(input2)

        # Process the third input through the fully connected layer
        input3 = self.fc_input3(input3)

        # Multiply the two outputs element-wise
        combined = input1 * input2 * input3

        # Pass the result through a fully connected layer
        combined = F.relu(self.fc_after_multiplication(combined))

        # Final output layer to produce a 174-dimensional output
        output = self.fc_output(combined)

        return output

def masked_spectral_distance(true, pred):
    """
    Calculate the masked spectral distance (similar to cosine distance) in PyTorch.
    """
    epsilon = 1e-8  # Small value to avoid division by zero

    # Masked predictions and true values (similar to the formula you provided)
    pred_masked = ((true + 1) * pred) / (true + 1 + epsilon)
    true_masked = ((true + 1) * true) / (true + 1 + epsilon)

    # Normalize both predicted and true masked tensors
    pred_norm = F.normalize(pred_masked, p=2, dim=-1)
    true_norm = F.normalize(true_masked, p=2, dim=-1)

    # Compute the cosine similarity (dot product)
    product = torch.sum(pred_norm * true_norm, dim=1)

    # Compute arccosine of the product (clip to avoid values outside the valid range for acos)
    product = torch.clamp(product, min=-1.0, max=1.0)
    arccos = torch.acos(product)

    # Return the normalized distance
    return 2 * arccos / np.pi

## xLSTM encoder architecture

In [ ]:
class xLSTMEncoderModel(nn.Module):
    def __init__(self):
        super(xLSTMEncoderModel, self).__init__()

        # Update embedding layer for input1 to handle values from 0 to 21 and output 32-dimensional embeddings
        self.embedding_input1 = nn.Embedding(22, 32)  # 22 because the values range from 0 to 21

        # xLSTM stack configuration
        cfg = xLSTMBlockStackConfig(
        mlstm_block=mLSTMBlockConfig(
        mlstm=mLSTMLayerConfig(
            conv1d_kernel_size=4, qkv_proj_blocksize=4, num_heads=4
        )
        ),
        slstm_block=sLSTMBlockConfig(
        slstm=sLSTMLayerConfig(
            backend="cuda",
            num_heads=4,
            conv1d_kernel_size=4,
            bias_init="powerlaw_blockdependent",
            ),
            feedforward=FeedForwardConfig(proj_factor=1.3, act_fn="gelu"),
            ),
            context_length=256,
            num_blocks=7,
            embedding_dim=32,  # Change embedding_dim to 32 to match the embedding output size
            slstm_at=[1],
            )

        # Instantiate the xLSTM block stack
        self.xlstm_stack = xLSTMBlockStack(cfg)

        # Fully connected layer for the first input (32-dimensional sequence)
        self.fc_input1 = nn.Linear(32, 124)

        # Fully connected layer for the second input (6-dimensional sequence)
        self.fc_input2 = nn.Linear(6, 124)

        # Fully connected layer for the third input (1-dimensional sequence)
        self.fc_input3 = nn.Linear(1, 124)

        # Fully connected layer after multiplication of both inputs
        self.fc_after_multiplication = nn.Linear(124, 124)

        # Final output layer to produce a 174-dimensional output
        self.fc_output = nn.Linear(124, 174)

    def forward(self, input1, input2, input3):
        # Process the first input through the embedding layer
        input1 = self.embedding_input1(input1.long())  # Ensure input1 is long for embedding

        # Pass the embedded input1 through the xLSTM stack
        # xLSTM expects [batch_size, seq_len, feature_dim]
        input1 = self.xlstm_stack(input1)  # [batch_size, seq_len, feature_dim]

        # Reduce input1 over the sequence length dimension (e.g., take sum)
        input1 = input1.sum(dim=1)  # Now shape is [batch_size, feature_dim]

        # Process the output of the xLSTM through the fully connected layer
        input1 = self.fc_input1(input1)

        # Process the second input through the fully connected layer
        input2 = self.fc_input2(input2)

        # Process the third input through the fully connected layer
        input3 = self.fc_input3(input3)

        # Multiply the two outputs element-wise
        combined = input1 * input2 * input3

        # Pass the result through a fully connected layer
        combined = F.relu(self.fc_after_multiplication(combined))

        # Final output layer to produce a 174-dimensional output
        output = self.fc_output(combined)

        return output
def masked_spectral_distance(true, pred):
    """
    Calculate the masked spectral distance (similar to cosine distance) in PyTorch.
    """
    epsilon = 1e-8  # Small value to avoid division by zero

    # Masked predictions and true values (similar to the formula you provided)
    pred_masked = ((true + 1) * pred) / (true + 1 + epsilon)
    true_masked = ((true + 1) * true) / (true + 1 + epsilon)

    # Normalize both predicted and true masked tensors
    pred_norm = F.normalize(pred_masked, p=2, dim=-1)
    true_norm = F.normalize(true_masked, p=2, dim=-1)

    # Compute the cosine similarity (dot product)
    product = torch.sum(pred_norm * true_norm, dim=1)

    # Compute arccosine of the product (clip to avoid values outside the valid range for acos)
    product = torch.clamp(product, min=-1.0, max=1.0)
    arccos = torch.acos(product)

    # Return the normalized distance
    return 2 * arccos / np.pi

## Select and compile the model

In [ ]:
# Hyperparameters
num_epochs = 50
learning_rate = 1e-3

# Initialize model, loss function, and optimizer
model = TransformerEncoderModel().to(device)  # Move model to CUDA
#model = xLSTMEncoderModel().to(device)  # Move model to CUDA

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

print(f'Number of trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}')


## Train the model

In [ ]:
train_losses = []  # To store training losses per epoch
test_losses = []   # To store evaluation losses per epoch

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    print(device)
    running_loss = 0.0  # To accumulate the training loss for this epoch

    for input1_batch, input2_batch, input3_batch, labels_batch in train_loader:
        optimizer.zero_grad()  # Zero the gradients

        # Move batches to CUDA (GPU) if available
        input1_batch, input2_batch, input3_batch, labels_batch = input1_batch.to(device), input2_batch.to(device), input3_batch.to(device), labels_batch.to(device)

        # Forward pass
        outputs = model(input1_batch, input2_batch, input3_batch)

        # Compute the loss using the masked spectral distance
        loss = masked_spectral_distance(labels_batch, outputs)

        # Accumulate the loss for the current batch
        running_loss += loss.mean().item()

        # Backward pass and optimization step
        loss.mean().backward()  # Backpropagate the mean loss
        optimizer.step()

    # Calculate the average training loss for the entire epoch
    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)  # Store the average training loss for this epoch

    # Now evaluate the model on the test set (validation set)
    model.eval()  # Set the model to evaluation mode
    running_test_loss = 0.0  # To accumulate the test loss

    with torch.no_grad():  # No need to compute gradients for evaluation
        for input1_batch, input2_batch, input3_batch, labels_batch in test_loader:
            # Move batches to CUDA (GPU) if available
            input1_batch, input2_batch, input3_batch, labels_batch = input1_batch.to(device), input2_batch.to(device), input3_batch.to(device), labels_batch.to(device)

            # Forward pass
            outputs = model(input1_batch, input2_batch, input3_batch)

            # Compute the loss using the masked spectral distance
            loss = masked_spectral_distance(labels_batch, outputs)

            # Accumulate the test loss for the current batch
            running_test_loss += loss.mean().item()

    # Calculate the average test loss for the entire epoch
    avg_test_loss = running_test_loss / len(test_loader)
    test_losses.append(avg_test_loss)  # Store the average test loss for this epoch

    print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

# Plotting the losses per epoch
plt.plot(range(1, num_epochs+1), train_losses, label='Training Loss', color='b')
plt.plot(range(1, num_epochs+1), test_losses, label='Validation Loss', color='r')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss per Epoch')
plt.legend()
plt.show()
